<a href="https://colab.research.google.com/github/laurenneal/capstone-visual-neuroscience/blob/Jordan/get_results_and_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Response and Stimulus

In [3]:
from google.colab import drive
drive.mount('/content/drive')

#imports
!pip install hdf5storage
import hdf5storage 
import pandas as pd
import numpy as np
from scipy import ndimage
from tqdm import notebook
from more_itertools import sliced
import matplotlib.pyplot as plt
import h5py

Mounted at /content/drive
     |████████████████████████████████| 53 kB 1.8 MB/s 


In [14]:
#This file contains the response and stimulus data for all stacks of this movie
#The file is named using the first stack in the file, but it contains the other stacks, 
#like 210815_0_2_stim_mc_mix1_syt_, 210815_0_3_stim_mc_mix1_syt_, etc, as well

path = '/content/drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded_CNMF/Stimulus_Features/210815_0_1_stim_mc_mix1_syt_.mat'
#read file in using hdf5 structure
results_file = hdf5storage.loadmat(path)

In [5]:
#list the two main folders
list(results_file.keys())

['resp', 'stim1']

In [6]:
#the response and stimulus are now in a 1x(number of stacks) matrix
results_file['resp'].shape

(1, 11)

In [7]:
#each stack has a (# of rois) by (# of frames) matrix holding the fluorescence trace of the rois in that stack
for x in results_file['resp'][0]:
  print(x.shape)

(15, 5286)
(15, 5286)
(15, 5286)
(15, 5286)
(15, 5286)
(15, 5286)
(15, 5286)
(15, 5286)
(15, 881)
(15, 3524)
(15, 5286)


In [8]:
#same for the stimulus values
for x in results_file['stim1'][0]:
  print(x.shape)

(15, 5286)
(15, 5286)
(15, 5286)
(15, 5286)
(15, 5286)
(15, 5286)
(15, 5286)
(15, 5286)
(15, 881)
(15, 3524)
(15, 5286)


In [9]:
! ls drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded_CNMF/Results

210815_0_10_stackRaw_mc_mix1_syt_result_20220213T070259.h5
210815_0_1_label_mc_mix1_syt_.csv
210815_0_1_stackRaw_mc_mix1_syt_result_20220213T070259.h5
210815_0_2_stackRaw_mc_mix1_syt_result_20220213T070259.h5
210815_0_3_stackRaw_mc_mix1_syt_result_20220213T070259.h5
210815_0_4_stackRaw_mc_mix1_syt_result_20220213T070259.h5
210815_0_5_stackRaw_mc_mix1_syt_result_20220213T070259.h5
210815_0_6_stackRaw_mc_mix1_syt_result_20220213T070259.h5
210815_0_7_stackRaw_mc_mix1_syt_result_20220213T070259.h5
210815_0_8_stackRaw_mc_mix1_syt_result_20220213T070259.h5
210815_0_9_stackRaw_mc_mix1_syt_result_20220213T070259.h5


Everything below this line needs to be reworked to run on the new format.

The spatial footprints are stored in the original 'results' folder that we need to path to, but the stimulus and response are stored together in the new format

Also need to use the filename, stack index, and roi index to create a unique identifier for each roi by stack

In [17]:
import os
root = 'drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded_CNMF/Results/'
spatial_files = os.listdir(root)
spatial_files = [s for s in spatial_files if s[-3:] == '.h5']
spatial_files

['210815_0_1_stackRaw_mc_mix1_syt_result_20220213T070259.h5',
 '210815_0_2_stackRaw_mc_mix1_syt_result_20220213T070259.h5',
 '210815_0_3_stackRaw_mc_mix1_syt_result_20220213T070259.h5',
 '210815_0_4_stackRaw_mc_mix1_syt_result_20220213T070259.h5',
 '210815_0_5_stackRaw_mc_mix1_syt_result_20220213T070259.h5',
 '210815_0_6_stackRaw_mc_mix1_syt_result_20220213T070259.h5',
 '210815_0_7_stackRaw_mc_mix1_syt_result_20220213T070259.h5',
 '210815_0_8_stackRaw_mc_mix1_syt_result_20220213T070259.h5',
 '210815_0_9_stackRaw_mc_mix1_syt_result_20220213T070259.h5',
 '210815_0_10_stackRaw_mc_mix1_syt_result_20220213T070259.h5']

In [33]:
rois = pd.DataFrame(columns=['stack','roi','spatial','temporal'])

In [35]:
h5py.File(root + spatial_files[0],'r')['spatial'][:].shape

(15, 32768)

In [36]:
h5py.File(root + spatial_files[0],'r')['temporal'][:].shape

(15, 5519)

In [ ]:
for f in enumerate(spatial_files): # the shapes above don't match up
  df = pd.DataFrame({
      'stack': f,
      'roi': , # use range to make a list for the numerb of rois
      'spatial': h5py.File(root + f,'r')['spatial'][:],
      'temporal': h5py.File(root + f,'r')['temporal'][:]
  })

In [ ]:
# stack of each roi's footprints - we'll use each of these
bwLabelStack = results_file['spatial']

bwLabelStack.shape

KeyError: ignored

In [15]:
results_file.keys()

dict_keys(['resp', 'stim1'])

In [ ]:
rois = 

In [37]:
# #plot the first roi
# dims = [128, 256]
# #save to variable
# spatial = bwLabelStack[0][:]
# plt.imshow(np.reshape(spatial, dims, order="F"))

In [38]:
# #visualize temporal traces - first roi
# temporal = results_file['temporal'][0]
# plt.figure(figsize=(20,5))
# plt.plot(temporal)

# Cell type labels

In [39]:
# import pandas as pd

# label_csv = pd.read_csv('/content/drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded_CNMF/Results/210815_0_1_label_mc_mix1_syt_.csv')
# label_csv

# Stimulus Data read and exploration

In [40]:
# stimulus_file = h5py.File(stimulus_path, 'r')
# stimulus_file['stimulus'].shape

In [41]:
# #plot the stimulus for the first roi for the first 200 frames
# #note that the matrix shape is flipped compared to the results matrices
# #need to access index zero with [:,0] rather than [0]
# plt.figure(figsize=(20,5))
# plt.plot(stimulus_file['stimulus'][:][:,0])

# Extract features from each roi in the file

In [42]:
# #IMPORTANT
# #Set the time period in frames that we want to consider here
# #If 1, then we're doing no temporal collapsing, just taking the fluorescence / stimulus
# #if >1, then we return the mean, min, max fluorescence over chunks of that length

# temporal_period_length = 1

# #every roi will have the same stack name, so declare it here
# stack_name = results_file['stack_name'][()].decode("utf-8")

In [43]:
# #set up different dataframes depending on if we're collpsing temporal features or not
# if temporal_period_length == 1:

#   roi_features = pd.DataFrame(columns =['roi_ID',
#                                       'area',
#                                       'center_of_mass_row',
#                                       'center_of_mass_column',
#                                       'fluorescence',
#                                       'stimulus_demo',
#                                       'label'])
  
# else:
#   roi_features = pd.DataFrame(columns =['roi_ID',
#                                     'area',
#                                     'center_of_mass_row',
#                                     'center_of_mass_column',
#                                     'mean_fluorescence',
#                                     'min_fluorescence',
#                                     'max_fluorescence',
#                                     'mean_stimulus_demo',
#                                     'min_stimulus_demo',
#                                     'max_stimulus_demo',
#                                     'label'])
  
# #loop through rois and extract features - these have not been validated to be calculated correctly
# for x in notebook.tqdm(range(len(bwLabelStack))):

#   spatial = bwLabelStack[x] #spatial footprint of the x'th roi
#   temporal = results_file['temporal'][x] #temporal trace of the x'th roi
#   stimulus = stimulus_file['stimulus'][:][:,x] #stimulus feature for the x'th roi
#   label = label_csv['label'][x] #cell type label of the x'th roi
#   roi_ID = stack_name + str(x) #comcatenated ID for the roi - date, fly index, stack index, mix, roi index

#   #calculate spatial measures
#   area = np.sum(spatial)
#   dims = [128,256]
#   spatial_img = np.reshape(spatial, dims, order="F")
#   center_of_mass_row, center_of_mass_column = ndimage.center_of_mass(spatial_img)
  
#   #If one sample is one frame, no temporal slicing necessary
#   if temporal_period_length == 1:

#     #to loop through all frames, swap the number for len(temporal)
#     for frame in notebook.tqdm(range(len(temporal))):

#       fluorescence = temporal[frame] #loop through frames
#       stimulus_value = stimulus[frame] #and loop through stimulus

#       roi_features = roi_features.append({'roi_ID': roi_ID,
#                                           'area': area,
#                                           'center_of_mass_row': center_of_mass_row,
#                                           'center_of_mass_column': center_of_mass_column,
#                                           'fluorescence': fluorescence,
#                                           'stimulus_demo': stimulus_value,
#                                           'label': label}, ignore_index=True)
      
#   else:

#     #list of lists containing the sliced up fluorescence values, one chunk has the length set in temporal_period_length
#     temporal_slices = list(sliced(temporal, temporal_period_length))
#     stimulus_slices = list(sliced(stimulus, temporal_period_length))

#     #loop through slices and extract collapsed temporal features for each slice
#     for temporal_slice in notebook.tqdm(range(len(temporal_slices))):

#       if len(temporal_slices[temporal_slice]) == temporal_period_length:

#         mean_fluorescence = np.mean(temporal_slices[temporal_slice]) #loop through frames
#         min_fluorescence = np.min(temporal_slices[temporal_slice])
#         max_fluorescence = np.max(temporal_slices[temporal_slice])

#         mean_stimulus = np.mean(stimulus_slices[temporal_slice]) #loop through frames
#         min_stimulus = np.min(stimulus_slices[temporal_slice])
#         max_stimulus = np.max(stimulus_slices[temporal_slice])

#         roi_features = roi_features.append({'roi_ID': roi_ID,
#                                             'area': area,
#                                             'center_of_mass_row': center_of_mass_row,
#                                             'center_of_mass_column': center_of_mass_column,
#                                             'mean_fluorescence': mean_fluorescence,
#                                             'min_fluorescence': min_fluorescence,
#                                             'max_fluorescence': max_fluorescence,
#                                             'mean_stimulus_demo': mean_stimulus,
#                                             'min_stimulus_demo': min_stimulus,
#                                             'max_stimulus_demo': max_stimulus,
#                                             'label': label}, ignore_index=True)


In [44]:
# roi_features.sample(10)

In [45]:
# len(roi_features)

In [ ]:
# close the .h5 files
results_file.close()
stimulus_file.close()

In [ ]:
#save the features - still needs validation
#manually set the filepath along with the number of frames in the window used
roi_features.to_csv('/content/drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded_CNMF/Extracted_Features/210815_0_1_label_mc_mix1_syt_features_1Frame_demo.csv')